In [2]:
# API KEY Loading
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_teddynote import logging

logging.langsmith("CH03-Prompt")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-Prompt


In [4]:
# LLM 객체 정의 
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

# Template 
## from_template

- from_template() 통해 PromptTemplate 생성 
- 변수들은 `{ }` 내에 포함

In [5]:
from langchain_core.prompts import PromptTemplate

In [6]:
# template 정의
template = "{country}의 언어는 무엇입니까?"

# PromptTemplate 객체 정의 
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 언어는 무엇입니까?')

In [7]:
# 변수 반영하여 프롬프트 정의 
prompt = prompt.format(country="영국")
prompt

'영국의 언어는 무엇입니까?'

In [8]:
# template 정의
template = "{country}의 언어는 무엇입니까?"

# PromptTemplate 객체 정의 
prompt = PromptTemplate.from_template(template)

# chain 정의 
chain = prompt | llm 

# 실행 
chain.invoke("영국")

AIMessage(content='영국의 주요 언어는 영어입니다. 그러나 영국 내에는 스코틀랜드, 웨일스, 북아일랜드 등의 지역에서 사용되는 지방어도 존재합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 25, 'total_tokens': 94, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fe757c9a-2e60-4384-9725-db6f8783c8ea-0', usage_metadata={'input_tokens': 25, 'output_tokens': 69, 'total_tokens': 94, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [9]:
chain.invoke("영국").content

'영국의 주요 언어는 영어입니다. 그러나 영국 내에는 웨일스어, 스코틀랜드 게일어, 스코틀랜드어, 아일랜드어와 같은 지방어도 사용됩니다.'

## PromptTemplate 

- `input_variables` : 명시적으로 지정하여 유효성 검사 진행 
- 정의된 내용과 다를 경우 예외 처리 

In [10]:
# template 정의 
template = "{country}의 언어는 무엇입니까?"

# PromptTemplate 객체 정의 
prompt = PromptTemplate(
    template=template,
    input_variables=['country']
)
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 언어는 무엇입니까?')

In [11]:
# # 변수 반영하여 프롬프트 완성
prompt.format(country="아일랜드")

'아일랜드의 언어는 무엇입니까?'

입력 변수가 2개인 경우...

`input_variables`에 dictionary 형태로 정의

In [12]:
# template 정의 
template = "{country1}과 {country2}의 언어는 각각 무엇입니까?"

# PromptTemplate 객체 정의 
prompt = PromptTemplate(
    template=template,
    input_variables=['country1'],
    partial_variables={
        "country2":"크로아티아",
    },
)
prompt

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '크로아티아'}, template='{country1}과 {country2}의 언어는 각각 무엇입니까?')

In [13]:
prompt.format(country1="대한민국")

'대한민국과 크로아티아의 언어는 각각 무엇입니까?'

In [14]:
prompt_partial = prompt.partial(country2="캐나다")
prompt_partial

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '캐나다'}, template='{country1}과 {country2}의 언어는 각각 무엇입니까?')

In [15]:
prompt_partial.format(country1="대한민국")

'대한민국과 캐나다의 언어는 각각 무엇입니까?'

In [16]:
chain = prompt_partial | llm
chain.invoke("대한민국").content

'대한민국의 언어는 한국어이고, 캐나다의 언어는 영어와 프랑스어입니다.'

In [17]:
chain.invoke(
    {
        'country1':'크로아티아',
        'country2':'슬로베니아',
    }
).content

'크로아티아어와 슬로베니아어입니다.'

`partial_variables`: 부분 변수 채움

- `partial` : 함수를 부분적으로 사용, **항상 공통된 방식으로 가져오고 싶은 변수** 가 있는 경우
- 대표적인 예 : **날짜나 시간** 

In [18]:
from datetime import datetime

# 오늘 날짜를 출력
datetime.now().strftime("%B %d")

'January 21'

In [19]:
# 날짜 반환 함수 정의
def get_today():
    return datetime.now().strftime("%B %d")

In [24]:
# PromptTemplate
prompt = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일과 직업을 함께 표기해주세요.",
    input_variables=["n"],
    partial_variables={
        "today": get_today  
    },
)

In [25]:
prompt.format(n=3)

'오늘의 날짜는 January 21 입니다. 오늘이 생일인 유명인 3명을 나열해 주세요. 생년월일과 직업을 함께 표기해주세요.'

In [26]:
# Chain
chain = prompt | llm

In [27]:
print(chain.invoke(3).content)

1. Emma Bunton
   - 생년월일: 1976년 1월 21일
   - 직업: 가수, 배우

2. Geena Davis
   - 생년월일: 1956년 1월 21일
   - 직업: 배우, 프로듀서, 모델

3. Placido Domingo
   - 생년월일: 1941년 1월 21일
   - 직업: 오페라 가수, 지휘자


In [29]:
# today 까지 지정하여 실행할 경우
print(
    chain.invoke(
        {
            'today':'1월 1일',
            'n': 3,
        }
    ).content
)

1. 김윤아 (1980년 1월 1일) - 가수
2. 조승우 (1979년 1월 1일) - 배우
3. 타이가 웨이티티 (1980년 1월 1일) - 미국의 배우


In [30]:
# 모델 버전 수정
llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-4o',
)

In [34]:
# today 까지 지정하여 실행할 경우
print(
    chain.invoke(
        {
            'today':'3월 1일',
            'n': 3,
        }
    ).content
)

1. 제이슨 테일러 (Jason Taylor)
   - 생년월일: 1974년 3월 1일
   - 직업: 프로 축구 선수 (NFL)
   
2. 저스틴 비버 (Justin Bieber)
   - 생년월일: 1994년 3월 1일
   - 직업: 가수, 싱어송라이터
   
3. 폴 린드 (Paul Hollywood)
   - 생년월일: 1966년 3월 1일
   - 직업: 셰프, TV 프로그램 진행자


## load_prompt

In [35]:
from langchain_core.prompts import load_prompt

In [37]:
prompt = load_prompt(
    'prompts/capital.yaml', 
    encoding='utf-8'
)
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디입니까?')

In [38]:
# 프롬프트 확인 
prompt.format(country='모로코')

'모로코의 수도는 어디입니까?'

In [40]:
prompt2 = load_prompt(
    'prompts/capital_detail.yaml', 
    encoding='utf-8'
)

print(prompt2.format(country='모로코'))

모로코의 수도에 대해서 알려주세요.
수도의 특징을 다음의 양식에 맞게 정리해 주세요.
300자 내외로 한글로 작성해 주세요.
----
[양식]
1. 면적
2. 인구
3. 역사적 장소
4. 특산품

#Answer:



In [41]:
# 스트리밍 실행 
from langchain_core.output_parsers import StrOutputParser
from langchain_teddynote.messages import stream_response

In [42]:
model = ChatOpenAI(
    temperature=0,
    model_name='gpt-4o',
)

chain = prompt2 | model | StrOutputParser()
answer = chain.stream('모로코')
stream_response(answer)

모로코의 수도는 라바트입니다. 

1. 면적: 라바트는 약 117km²의 면적을 가지고 있으며, 대서양 연안에 위치해 있어 해안 도시로서의 매력을 지니고 있습니다.

2. 인구: 라바트의 인구는 약 58만 명으로, 모로코의 정치적 중심지로서 다양한 인구가 거주하고 있습니다.

3. 역사적 장소: 라바트에는 유네스코 세계문화유산으로 지정된 우다이아 카스바와 하산 타워가 있습니다. 이곳은 모로코의 역사와 문화를 엿볼 수 있는 중요한 유적지입니다.

4. 특산품: 라바트는 전통적인 모로코 카펫과 수공예품으로 유명합니다. 특히, 정교한 디자인과 색상이 돋보이는 카펫은 많은 관광객들에게 인기가 있습니다.

# ChatPromptTemplate

- `PromptTemplate` : 일반 텍스트 기반의 프롬프트 생성
- `ChatPromptTemplate` : 메시지 기반 프롬프트 생성 
    - 여러 역할(role)을 포함한 메시지 체인 생성 (`system`,`human`,`ai`)
        - `"system"`: 시스템 설정 메시지. 주로 전역설정(eg, 페르소나, 임무)과 관련
        - `"human"` : 사용자 입력 메시지
        - `"ai"`: AI 의 답변 메시지
    - 복잡한 Context나 멀티턴 대화 관리 

- 주요 특징 비교

| **특징**                | **PromptTemplate**                         | **ChatPromptTemplate**                  |
|-------------------------|--------------------------------------------|-----------------------------------------|
| **형식**                | 단일 텍스트 기반 프롬프트                  | 대화형 메시지 체인                      |
| **사용 모델**           | 텍스트 입력 기반 모델 (예: GPT-3)           | 대화 기반 모델 (예: ChatGPT)            |
| **복잡성 처리**         | 간단한 프롬프트 처리                        | 여러 역할과 메시지 간 대화 시나리오 처리 |
| **역할 지원**           | 지원하지 않음                              | `system`, `user`, `assistant` 지원      |
| **생성 방식**           | `.format()`으로 텍스트 생성                 | `.format_messages()`으로 메시지 생성    |


In [43]:
from langchain_core.prompts import ChatPromptTemplate

In [44]:
prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디입니까?")
prompt

ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디입니까?'), additional_kwargs={})])

In [45]:
prompt.format(country='캐나다')

'Human: 캐나다의 수도는 어디입니까?'

In [47]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ('system', '친절한 AI Assistant 입니다. 당신의 이름은 {name} 입니다.'),
        ('human', '반가워요!'),
        ('ai', '안녕하세요! 무엇을 도와드릴까요?'),
        ('human', "{user_input}"),
    ]
)

In [50]:
messages = chat_template.format_messages(
    name="철수",
    user_input="서울시청 전화번호는 무엇인가요?",
)

messages

[SystemMessage(content='친절한 AI Assistant 입니다. 당신의 이름은 철수 입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='서울시청 전화번호는 무엇인가요?', additional_kwargs={}, response_metadata={})]

In [51]:
llm.invoke(messages).content

'서울시청의 대표 전화번호는 02-120입니다. 이 번호로 전화를 걸면 서울시의 다양한 서비스에 대한 안내를 받을 수 있습니다. 추가로 궁금한 점이 있으면 말씀해 주세요!'

In [52]:
# Chain
chain = chat_template | llm 
chain.invoke(
    {
        'name':'철수',
        'user_input':'서울시청 전화번호는 무엇인가요?',
    }
).content

'서울시청의 대표 전화번호는 02-120입니다. 이 번호로 전화를 걸면 서울시의 다양한 서비스에 대한 안내를 받을 수 있습니다. 추가로 궁금한 점이 있으면 말씀해 주세요!'

# MessagePlaceholder

In [53]:
 from langchain_core.prompts import MessagesPlaceholder

In [54]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system','요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.'
        ),
        MessagesPlaceholder(variable_name='conversation'),
        ('human', '지금까지 대화를 {word_count} 단어로 요약합니다.')
    ]
)

chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annota

In [55]:
chat_prompt_formatted = chat_prompt.format(
    word_count=5,
    conversation=[
        ('human', '안녕하세요! 오늘 처음 인사드리는 김영희 입니다. 만나서 반갑습니다. 여러분들과 1기수를 즐겁고 행복하게 지내면 좋겠습니다.'),
        ('ai', '반갑습니다! 저도 앞으로 잘 부탁드립니다.'),
    ]
)

print(chat_prompt_formatted)

System: 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.
Human: 안녕하세요! 오늘 처음 인사드리는 김영희 입니다. 만나서 반갑습니다. 여러분들과 1기수를 즐겁고 행복하게 지내면 좋겠습니다.
AI: 반갑습니다! 저도 앞으로 잘 부탁드립니다.
Human: 지금까지 대화를 5 단어로 요약합니다.


In [56]:
# Chain 
chain = chat_prompt | llm | StrOutputParser()

In [58]:
# Chain 실행 
chain.invoke(
    {
        'word_count': 5,
        'conversation': [
            ('human', '안녕하세요! 오늘 처음 인사드리는 김영희 입니다. 만나서 반갑습니다. 여러분들과 1기수를 즐겁고 행복하게 지내면 좋겠습니다.'),
            ('ai', '반갑습니다! 저도 앞으로 잘 부탁드립니다. AI를 공부하시는데 도움 될 수 있도록 노력하겠습니다.'),
        ]
    }
)

'김영희, 첫 인사, 반가움.'

-----
** End of Documents **